In [0]:
users_data_df = spark.read.format("delta").load("/mnt/adls21s/silver/users_data/")
display(users_data_df)
cards_data_df = spark.read.format("delta").load("/mnt/adls21s/silver/cards_data/")
display(cards_data_df)
transactions_data_df = spark.read.format("delta").load("/mnt/adls21s/silver/transactions_data/")
display(transactions_data_df)

In [0]:
from pyspark.sql.functions import year, month, day, hour, minute, second, col, date_diff, current_date

#Creating dimension date table
dim_date = transactions_data_df.select("transaction_id", year(("date")), month(("date")), day(("date")), hour(("date")), minute(("date")))
dim_date = dim_date.withColumnRenamed("year(date)", "year").withColumnRenamed("month(date)", "month").withColumnRenamed("day(date)", "day").withColumnRenamed("hour(date)", "hour").withColumnRenamed("minute(date)", "minute")

#Creating dimesion user table
dim_users = users_data_df.select("user_id", "current_age", "retirement_age", "birth_year", "birth_month", "gender", "credit_score", "num_credit_cards", "per_capita_income_in_$", "yearly_income_in_$", "total_debt_in_$")

#Creating dimesion card table
dim_cards = cards_data_df.select("card_id", col("client_id").alias("user_id"), "card_brand", "card_type", year("expires").alias("year"), month("expires").alias("month"), "has_chip", "num_cards_issued", "acct_open_date", "year_pin_last_changed", "card_on_dark_web", "credit_limit_in_$").withColumn("acct_age_yrs", date_diff(current_date(), col("acct_open_date"))/365)

#Creating dimesion merchant table
dim_merchants = transactions_data_df.select("merchant_id", "merchant_city", "merchant_state", "zip", "mcc")

#Creating fact transaction table
fact_transactions = transactions_data_df.select("transaction_id", col("client_id").alias("user_id"), "card_id", "use_chip", "merchant_id", "amount_in_$", "errors")


In [0]:
transactions = display(fact_transactions.join(dim_date,"transaction_id","right"))